In [3]:
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import applications

In [4]:
vgg=applications.vgg16.VGG16()

In [9]:


vgg.summary()



Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
cnn=Sequential()
for capa in vgg.layers:
    cnn.add(capa)

In [11]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [6]:
cnn.pop()

In [13]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [7]:


for layer in cnn.layers:
    layer.trainable=False



In [8]:
cnn.add(Dense(3,activation='softmax'))

In [16]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [15]:
def modelo():
    vgg=applications.vgg16.VGG16()
    cnn=Sequential()
    for capa in vgg.layers: cnn.add(capa)
    cnn.layers.pop()
    for layer in cnn.layers: layer.trainable=False
    cnn.add(Dense(10,activation='softmax'))
    return cnn

In [16]:
K.clear_session()

data_entrenamiento = './Dataset/train'
data_validacion = './Dataset/test'

epocas=10
longitud, altura = 224, 224
batch_size = 1
pasos = 200
validation_steps = 100
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 10
lr = 0.0005


##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.5,
    zoom_range=0.5,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

##CREAR LA RED VGG16

cnn=modelo()

cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])


cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)



Found 1413 images belonging to 10 classes.
Found 448 images belonging to 10 classes.
Epoch 1/10
200/200 [==============================] - 402s 2s/step - loss: 2.2956 - accuracy: 0.1950 - val_loss: 2.2958 - val_accuracy: 0.1000
Epoch 2/10
200/200 [==============================] - 411s 2s/step - loss: 2.2883 - accuracy: 0.1550 - val_loss: 2.2918 - val_accuracy: 0.1300
Epoch 3/10
200/200 [==============================] - 530s 3s/step - loss: 2.2795 - accuracy: 0.1650 - val_loss: 2.2785 - val_accuracy: 0.1800
Epoch 4/10
200/200 [==============================] - 462s 2s/step - loss: 2.2737 - accuracy: 0.1550 - val_loss: 2.2852 - val_accuracy: 0.1200
Epoch 5/10
200/200 [==============================] - 460s 2s/step - loss: 2.2739 - accuracy: 0.1400 - val_loss: 2.2647 - val_accuracy: 0.1900
Epoch 6/10
200/200 [==============================] - 420s 2s/step - loss: 2.2663 - accuracy: 0.1350 - val_loss: 2.2613 - val_accuracy: 0.1900
Epoch 7/10
200/200 [==============================] - 433

In [17]:
target_dir = './modelox/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelox/modelo.h5')
cnn.save_weights('./modelox/pesos.h5')